In [1]:
import os 
import sys
import re
import time
import csv
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import ik_parsing as ik
import data_extraction as de
import helper_functions as hp
import defs
from datetime import datetime
from collections import defaultdict
import itertools
import zipfile
import gzip
import json
import concurrent.futures
from multiprocessing import Pool
import matplotlib.pyplot as plt
import math

head, tail = os.path.split(os.getcwd())

In [4]:
head, tail = os.path.split(os.getcwd())
# list_of_filenames = ['158995321.txt']
list_of_filenames = os.listdir(os.path.join(head, 'sample_cases_10k'))
print(len(list_of_filenames))
list_of_banks = pd.read_csv('../bank_names.csv', header=None)
list_of_banks = list_of_banks[0].tolist()
list_of_banks = ik.preprocess_company_names(list_of_banks, bank=True)
list_of_banks[:5]

9923


['axis bank', 'bandhan bank', 'csb bank', 'city union bank', 'dcb bank']

In [7]:
import importlib
importlib.reload(ik)
importlib.reload(de)


files = []
titles = []
dates = []
courts = []
cnr_nums = []
case_nums = []
petitioner = []
respondent = []
petitioner_advocates = []
respondent_advocates = []
judges = []
banks = []
citations = []
acts = []
count = 0
for filename in list_of_filenames:
#     count += 1
#     if count > 5:
#         break
    with open(os.path.join(os.path.join(head, 'sample_cases_10k', filename)), encoding="utf8") as f:
        soup = BeautifulSoup(f, 'html.parser')
        citation_banner = soup.find('div', 'doc_cite')
        if citation_banner:
            citation_banner.extract()
        text = soup.get_text().lower()
#         print(soup.find_all('a'))
        acts_set = set(re.findall("""/doc/(\d*).*(?:section|constitution|penal|act).*<""", str(soup), flags=re.IGNORECASE))
        for act in acts:
            pass
#             print(act)
#         soup.extract('div', 'doc_cite')
        files.append(filename)
        try:
            title, date = soup.title.string.split(' on ')
            date = datetime.strptime(date, "%d %B, %Y")
            titles.append(title)
            dates.append(date)
        except (AttributeError, ValueError):
            titles.append(np.nan)
            dates.append(np.nan)
        try:
            courts.append(soup.find("div", {"class": "docsource_main"}).text)
        except AttributeError:
            courts.append(np.nan)
        case_nums.append(ik.extract_case_num(text))
        petitioner.append(ik.extract_petitioner(text))
        respondent.append(ik.extract_respondent(text))
        pet_adv, resp_adv = ik.extract_petitioner_advocate(text)
        petitioner_advocates.append(pet_adv)
        respondent_advocates.append(resp_adv)
        cnr_nums.append(ik.extract_cnr(text))
        judges.append(ik.extract_judges(text, soup))
        banks.append(ik.extract_banks(text, list_of_banks))
        citations_cited, acts_cited = ik.extract_citations(filename, str(soup))
        citations.append(citations_cited)
        acts.append(acts_cited)
        
df = pd.DataFrame({'file': files, 'cnr_num': cnr_nums, 'title': titles, 'date': dates,
                   'court': courts, 'case_number': case_nums, 'petitioner': petitioner, 'respondent': respondent,
                   'petitioner_advocate': petitioner_advocates, 'respondent_advocate': respondent_advocates,
                   'judge': judges, 'banks': banks, 'cases_cited': citations, 'acts_cited': acts})
df

,file,cnr_num,title,date,court,case_number,petitioner,respondent,petitioner_advocate,respondent_advocate,judge,banks,cases_cited,acts_cited
0,100002819.txt,NaN,M/S Supreme Brahmaputra (Jv) vs The State Of B...,2015-12-22,Patna High Court - Orders,civil writ jurisdiction case no.10232 of 2015,,,NaN,NaN,{mr. justice vikash jain},NaN,{152515230},{}
1,1000294.txt,NaN,Mirza Nausherwan Khan & Anr vs The Collector (...,1974-09-26,Supreme Court of India,NaN,NaN,NaN,NaN,NaN,"{Krishnaiyer, V.R.}",NaN,"{780609, 1251571}",{7832}
2,100092195.txt,NaN,Sri Nirakar Sahoo vs Sutar Chemical Private Lt...,2019-08-01,Supreme Court - Daily Orders,NaN,,,NaN,NaN,NaN,NaN,{},{}
3,1000974.txt,NaN,Shiv Ji Sharma vs The State Of Bihar & Ors,2011-10-20,Patna High Court - Orders,NaN,,,NaN,NaN,"{sheema ali khan, j.}",NaN,{},{1965344}
4,100097838.txt,NaN,Divisional Manager National ... vs Satya Narai...,2011-01-13,Patna High Court - Orders,NaN,,,NaN,NaN,"{anjana prakash, j.}",NaN,{},{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9918,99931859.txt,\ngahc010025502019\n,Nurjahan Begum vs The Union Of India And 5 Ors,2019-02-22,Gauhati High Court,case no. : wp(c) 1087/2019,1:nurjahan begum\n d/o- late hussenkha...,\n\n 1:the union of india and 5 ors\n ...,: mr. u c rabha\n\n,asstt.s.g.i.\n\n\n\n\n ...,{mr. justice achintya malla bujor barua ...,NaN,{},{}
9919,99946569.txt,NaN,Sandeep vs State Of Haryana And Others,2019-09-23,Punjab-Haryana High Court,NaN,,\n\n\nstate of haryana and others\n\n ...,,NaN,{mr. justice rajiv narain raina},NaN,{},{}
9920,99965171.txt,NaN,Kanhaiya Bhagat vs The State Of Bihar,2012-09-12,Patna High Court - Orders,in connection with darauli p.s. case no. 58 of...,,,NaN,NaN,{mr. justice hemant kumar},NaN,{},{}
9921,99965711.txt,NaN,I.D. Kamboj vs Mrs. Indira Bhardwaj And Others,2012-02-28,Punjab-Haryana High Court,NaN,NaN,NaN,,NaN,{mr.justice l.n. mittal},{'punjab national bank': 1},{1331149},{}


In [13]:
df.citations[0]

{'1085825',
 '1251571',
 '1288162',
 '1861446',
 '780609',
 '7832',
 '859367',
 '979365'}

In [12]:
c = {1,2,3,4}
d = {3,4}
c = c-d

In [13]:
c

{1, 2}